In [ ]:

import numpy as np
from pathlib import Path
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
keyFileNameCorr = '/content/drive/MyDrive/Morteza/LungProject/PatientFilenameOrganizaiton/Lung1-7.xlsx'
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from collections import Counter
from sklearn.datasets import make_classification
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from matplotlib import pyplot
from numpy import where
from statistics import mean
from statistics import stdev
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
import numpy as np
from sklearn.decomposition import MiniBatchSparsePCA
from imblearn.over_sampling import RandomOverSampler 
from random import sample
clinical_416_address = '/content/drive/MyDrive/Morteza/LungProject/PatientFilenameOrganizaiton/416_Prediction_Clinical_variables.xlsx'
df_416 = pd.read_excel(clinical_416_address, dtype=str)
df_416['SexGroup'].replace(['Male', 'Female'],[0, 1], inplace=True)
df_416['StageCat'].replace(['Stage1-2', 'Stage3_4'],[0, 1], inplace=True)
df_key_filename = pd.read_excel(keyFileNameCorr, dtype=str)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

random_state_num = 22


In [ ]:


# label_column = df_416['Adenocarcinoma.predominant.type.1.lepidic.2.Acinar.3.Papillary.4.Micropapillary.5.Solid'].values
# label_column = df_416['SexGroup'].values
# label_column = df_416['StageCat'].values  #(comes with nan values)
# label_column = df_416['Progression (0 = no, 1= yes)'].values #(comes with nan values)
# label_column = df_416['BMI <30 0, <30 1'].values
label_column = df_416['Smoker 0=never 1 = smoker'].values
# label_column = df_416['Age above (1) below (0) 70'].values
# label_column = df_416['Age above (1) below (0) 75'].values
# label_column = df_416['survival 3 years'].values
# label_column = df_416['5 year survival'].values

file_name_column = df_416['File.name'].values



used = set()
# mylist = [u'nowplaying', u'PBS', u'PBS', u'nowplaying', u'job', u'debate', u'thenandnow']
unique = [x for x in label_column if x not in used and (used.add(x) or True)]
import math
unique = [x for x in unique if not (isinstance(x, float) and math.isnan(x))]

print(unique)
y = []
X = []

target_0 = unique[0]
target_1 = unique[1]


# target_0 = '0'
# target_1 = '1'
if len(unique) > 2:
  target_2 = unique[2]
  target_3 = unique[3]
  target_4 = unique[4]

['1', '0']


In [ ]:
cell_freq = pd.read_csv('/content/drive/MyDrive/Morteza/LungProject/CellFrequencies/cell_frequencies.csv', dtype=str)

In [ ]:
cellFrequencies = cell_freq.iloc[:,range(4,38,2)].values
cellFrequencies = np.asarray(cellFrequencies, dtype=np.float64, order='C')

In [ ]:
allFileNames = cell_freq['File name']


In [ ]:
X = []
y = []
# print(len(unique))



def remove_end_spaces(string):
    return "".join(string.rstrip())
for i in range(416):
  ind_num = [index for index, value in enumerate(allFileNames[:416]) if value == remove_end_spaces(file_name_column[i])]
  
  y_val = label_column[i]
  # print(ind_num)

  if len(unique)==2:
    if y_val==target_0 or y_val==target_1:    
      
      if len(X)==0 and len(ind_num)!=0:
        y.append(y_val)
        X = cellFrequencies[ind_num,:]
        
      elif len(ind_num)!=0:
        y.append(y_val)
        X = np.vstack([X, cellFrequencies[ind_num,:]])
  elif len(unique) > 2:
    if y_val==target_0 or y_val==target_1 or y_val==target_2 or y_val==target_3 or y_val==target_4:    
      
      if len(X)==0 and len(ind_num)!=0:
        y.append(y_val)
        X = cellFrequencies[ind_num,:]
      elif len(ind_num)!=0:
        y.append(y_val)
        X = np.vstack([X, cellFrequencies[ind_num,:]])


copyX = X
copyY = y

In [ ]:
X = copyX
y = copyY
A = Counter(y)
if len(unique)==2:
  class_0 = A.get(target_0)
  class_1 = A.get(target_1)
  n0 = int(class_0/10)
  n1 = int(class_1/10)
  baseline_dist = max(class_0,class_1)/(max(class_0,class_1)+min(class_0,class_1))
  ratio = max(class_0,class_1)/min(class_0,class_1)
elif len(unique) > 2:
  ratio = 2
print('Original dataset shape %s' % Counter(y))
if ratio >=2:
  ros = RandomOverSampler(random_state=42)
  X, y = ros.fit_resample(X, y)
  print('Resampled dataset shape %s' % Counter(y))


yy = np.array(y)
import numpy as np
from sklearn.model_selection import KFold
kf = KFold(n_splits=5,random_state=10,shuffle=True)
kf.get_n_splits(X)

delete_dup = True
# print(kf)

for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = yy[train_index], yy[test_index]
    toBeDeleted = []
    if delete_dup: 
      for i in range(X_test.shape[0]):
        query = X_test[i]
        j = 0
        query_found = 0
        while j < X_train.shape[0] and query_found==0:
          match = X_train[j]
          # print(query)
          dist = np.linalg.norm(query-match)
          if dist == 0:
            query_found = 1
            toBeDeleted.append(i)
          j = j + 1

      if X_test.shape[0]-len(toBeDeleted) >= 5:
        X_test = np.delete(X_test,np.s_[toBeDeleted],axis=0)
        y_test = np.delete(y_test,np.s_[toBeDeleted],axis=0)
    transformer = MiniBatchSparsePCA(n_components=10, batch_size=500,random_state=0)
    transformer.fit(X_train)
    X_tr_em = transformer.transform(X_train)
    X_te_em = transformer.transform(X_test)
    kernel = 1.0 * RBF(1.0)
    clf = make_pipeline(StandardScaler(), SVC(gamma=1e1,C=1e1,kernel = kernel))
    clf.fit(X_tr_em, y_train)
    print('test score = '+str(clf.score(X_te_em,y_test)*100))
    
A = Counter(copyY)
if len(unique)==2:
  class_0 = A.get(target_0)
  class_1 = A.get(target_1)
  ratio = max(class_0,class_1)/(max(class_0,class_1)+min(class_0,class_1))
  print('baseline = '+str(ratio))


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

Original dataset shape Counter({'1': 377, '0': 38})
Resampled dataset shape Counter({'1': 377, '0': 377})
test score = 92.20779220779221
test score = 96.1038961038961
test score = 93.05555555555556
test score = 94.8051948051948
test score = 93.42105263157895
baseline = 0.908433734939759
